Corriger une prédiction HTR
===
<br>
D'après la démarche expliquée dans la documentation du projet DAHN, on procède selon les étapes suivantes.

On privilégie le format XML-Page pour exporter et réimporter les prédictions HTR dans le logiciel de transcription.

In [1]:
from py.constantes import XMLaCORRIGER, XMLCORRIGEES
print(f"On place les prédictions HTR dans le dossier : {XMLaCORRIGER}")

On place les prédictions HTR dans le dossier : ./xmlPage-aCorriger/


# Analyser le contenu de la prédiction

Exécuter le script **spellcheckTexts.py**, qui analyse chaque mot des prédictions HTR et génère pour chaque fichier XML à corriger un dictionnaire au format Json contenant des propositions de corrections (cela peut prendre quelques minutes…) : 

In [1]:
!python3 py/spellcheckTexts.py

Le fichier ./xmlPage-aCorriger/CdS02_Konv002-03_0058.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/aCorriger/page_CdS02_Konv002-03_0058.json a été écrit correctement.



Il est conseillé de placer plusieurs pages à corriger ensemble dans le dossier. Le temps de calcul étant long (5 min environ), on peut ainsi commencer à corriger la première page terminée tandis que le script continue à tourner sur les autres.

# Corriger les dictionnaires de pages

Corriger à la main les entrées du dictionnaire de chacun des fichiers commençant par "page" générés dans le dossier [dicos/aCorriger](py/dicos/aCorriger).

## Forme erronée et plusieurs propositions de correction

Lorsque les propositions de corrections sont multiples, on doit veiller à placer en première position la correction souhaitée :
```json
"fout": {
    "lem": [
        "faut",
        "font"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```

Il faut donc corriger pour obtenir :
```json
"fout": {
    "lem": [
        "font",
        "faut"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```

On peut également supprimer la correction non souhaitée :
```json
"fout": {
    "lem": [
        "font"
    ],
  "ctxt": ["Les bons comptes FOUT les bons amis"]
```


## Forme juste et proposition de correction fausse

Lorsque la forme prédite est juste mais que la proposition de correction n'est pas juste, il faut remplacer cette dernière par la forme juste. Par exemple, le dictionnaire propose automatiquement :
```json
"Cet": {
    "lem": [
        "et"
        ],
    "ctxt": "CET ouvrage que je viens de publier"
}
```
Ici, "Cet" est juste, et la proposition "et" est fausse, on doit donc la remplacer ainsi :
```json
"Cet": {
    "lem": [
        "Cet"
        ],
    "ctxt": "CET ouvrage que je viens de publier"
}
```

## Forme juste et proposition de correction "null"

Il n'est pas nécessaire de transformer les mots justes dont le lemme est indiqué comme "null".

## Les noms propres

Les **patronymes** représentent une difficulté importante de la transcription. Ils ont par bonheur été déjà identifiés. On se reportera donc aux notices du site web. On se reportera aux [outils du dossier donnees/](https://github.com/sbiay/CdS-edition/tree/main/donnees#outils), et plus particulièrement au [notebook](https://github.com/sbiay/CdS-edition/blob/main/donnees/Obtenir_metadonnees_images.ipynb) dédié à la récupération des liens vers les notices de l'inventaire à partir d'un nom de fichier image.

D'une façon générale, il est peu efficace de proposer des corrections pour les mots brefs et rencontrés dans **les passages les plus cursifs** (bouts de lignes, date, expéditeur) : la pércision de la reconnaissance y est faible, et plus un mot et bref plus la correction pose de problèmes d'ambiguïté.

A l'inverse, il n'est pas gênant de proposer des corrections pour les mots très mal prédits, longs, coupés par une espace. L'intérêt d'inscrire cette correction dans le dictionnaire est limité, car la probabilité de recroiser la même forme est à peu près nulle, mais le risque d'ambiguïté est tout aussi faible : autant appliquer la correction dès le stade de la validation du dictionnaire de page plutôt que d'attendre de la recroiser à la relecture finale.

## Sauvegarder le dictionnaire de page corrigé

Une fois les corrections faites, on copie le fichier dans le dossier [dicos/corriges](py/dicos/corriges).

# Appliquer le dictionnaire du projet aux fichiers XML

On procède grâce au script [text_correction_XML.py](./py/text_correction_XML.py) :

In [4]:
!python3 py/textCorrection.py

et
Traceback (most recent call last):
  File "py/textCorrection.py", line 160, in <module>
    textCorrectionXML()
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/sbiay/dhi/venv/lib/python3.8/site-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "py/textCorrection.py", line 140, in textCorrectionXML
    if not entreesMAJ[str(entree)]["lem"] in dictCDS[str(entree)]["lem"]:
TypeError: list indices must be integers or slices, not str


Ce script ajoute automatiquement les corrections appliquées au fichier global [correctionsCDS.json](./py/dicos/correctionsCDS.json). Si d'autres corrections existent déjà pour les mêmes formes dans [correctionsCDS.json](./py/dicos/correctionsCDS.json), l'ensemble des corrections constituent une liste, réexploitée à la prochaine correction de page en proposant comme premier choix le mot le plus fréquent parmi les vérités de terrain.

On finalise le processus de correction avec les étapes suivantes :
1. Réimporter les fichiers corrigés dans eScriptorium ;
2. Finaliser manuellement la correction des transcriptions ;
3. Télécharger la verité de terrain pour la sauvegarder au format XML-Alto.

In [ ]:
print(f"On place les vérités de terrain dans le dossier : {XMLCORRIGEES}")